【Python】トピックモデル（LDA）


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Pythonのgensimの中にLDAのライブラリがあるので、これを使えば手軽にトピックモデルを試すことができます。
事前に用意するのは、一つのテキストデータを一行としたtrain.txtとtest.txtのみです。

クラスタリングは教師なし学習なのでトレーニンデータとテストデータに分ける必要はないのですが、公式サイトではトレーニンデータとテストデータに分けていたのでそれに倣いました。
あと、適当なテキストデータがないためヤフーのトピックス一覧から持ってきました。実際に手元で試す際は分析したいテキストデータに置き換えて下さい。

In [ ]:
!pip install mecab-python3
!pip install unidic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7405 sha256=8c6553488022b3b1d7e649e48deccb12f90835e5bc8c726da634cf252e9df09d
  Stored in directory: /root/.cache/pip/wheels/7a/72/72/1f3d654c345ea69d5d51b531c90daf7ba14cc555eaf2c64ab0
Successfully built unidic
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2


In [ ]:
# MeCabを使うときにここ入れる
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30
Suggested packages:
  apt-xapian-index aptitude-doc-en | aptitude-doc debtags tasksel libcwidget-dev xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30
0 upgraded, 5 newly installed, 0 to remove and 23 not upgraded.
Need to get 3,838 kB of archives.
After this operation, 17.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aptitude-common all 0.8.13-3ubuntu1 [1,719 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsigc++-2.0-0v5 amd64 2.10.4-2ubuntu3 [12.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcwidget4 amd64 0.5.18-5build1 [306 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libxapian30 amd64 1

ここでランタイム再起動する

In [ ]:
import MeCab
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from collections import defaultdict


In [ ]:
# MeCabオブジェクトの生成
mt = MeCab.Tagger('-Owakati')
mt.parse('text')

# トピック数の設定
NUM_TOPICS = 3

In [ ]:

if __name__ == "__main__":
    # トレーニングデータの読み込み
    # train_texts は二次元のリスト
    # テキストデータを一件ずつ分かち書き（名詞、動詞、形容詞に限定）して train_texts に格納するだけ
    train_texts = []
    with open('/content/LDA_train.txt', 'r') as f:
        for line in f:
            text = []
            node = mt.parseToNode(line.strip())
            while node:
                fields = node.feature.split(",")
                if fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞':
                    text.append(node.surface)
                node = node.next
            train_texts.append(text)

    # モデル作成
    dictionary = Dictionary(train_texts)
    corpus = [dictionary.doc2bow(text) for text in train_texts]
    lda = LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

    # テストデータ読み込み
    # test_texts は train_texts と同じフォーマット
    test_texts = []
    raw_test_texts = []
    with open('/content/LDA_test.txt', 'r') as f:
        for line in f:
            text = []
            raw_test_texts.append(line.strip())
            node = mt.parseToNode(line.strip())
            while node:
                fields = node.feature.split(",")
                if fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞':
                    text.append(node.surface)
                node = node.next
            test_texts.append(text)

    # テストデータをモデルに掛ける
    score_by_topic = defaultdict(int)
    test_corpus = [dictionary.doc2bow(text) for text in test_texts]

    # クラスタリング結果を出力
    for unseen_doc, raw_train_text in zip(test_corpus, raw_test_texts):
        print(raw_train_text, end='\t')
        for topic, score in lda[unseen_doc]:
            score_by_topic[int(topic)] = float(score)
        for i in range(NUM_TOPICS):
            print('{:.2f}'.format(score_by_topic[i]), end='\t')
        print()

爆発 現場で缶100本ガス抜き	0.33	0.33	0.33	
爆発の瞬間 デマ動画が拡散	0.33	0.33	0.33	
バニラエア 来年10月運航終了	0.33	0.33	0.33	
コンビニごみ 店舗負担の矛盾	0.33	0.33	0.33	
謎めいたカエル 南米で再発見	0.33	0.33	0.33	
イニエスタ うつ報道の難しさ	0.33	0.33	0.33	
ムネリン笑顔「少し元気に」	0.33	0.33	0.33	
G菅野 ゴジラ超え6.5億円に	0.33	0.33	0.33	
